In [1]:
import shutup

shutup.please()

import pandas as pd
import numpy as np
from datetime import datetime
import os, os.path, pickle, sys
import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

from sklearn import preprocessing
import statistics
import statsmodels.api as sm

sys.path.append("/Users/hn/Documents/00_GitHub/Rangeland/Python_Codes/")
import rangeland_core as rc

In [2]:
data_dir_base = "/Users/hn/Documents/01_research_data/RangeLand/Data/"
census_population_dir = data_dir_base + "census/"
# Shannon_data_dir = data_dir_base + "Shannon_Data/"
NASS_dir = data_dir_base + "/NASS_downloads/"
param_dir = data_dir_base + "parameters/"
Min_data_base = data_dir_base + "Min_Data/"
reOrganized_dir = data_dir_base + "reOrganized/"

In [3]:
slaughter_dir = NASS_dir + "slaughter_Qs/"

In [4]:
slaughter_Q1 = pd.read_csv(slaughter_dir + "slaughter_Q1.csv")
slaughter_Q1.rename(columns=lambda x: x.lower().replace(' ', '_'), inplace=True)
slaughter_Q1.head(2)

,program,year,period,week_ending,geo_level,state,state_ansi,ag_district,ag_district_code,county,...,zip_code,region,watershed_code,watershed,commodity,data_item,domain,domain_category,value,cv_(%)
0,CENSUS,2017,YEAR,NaN,COUNTY,ALASKA,2,TANANA VALLEY,10,FAIRBANKS NORTH STAR,...,NaN,NaN,0,NaN,CATTLE,"CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURE...",TOTAL,NOT SPECIFIED,(D),(D)
1,CENSUS,2017,YEAR,NaN,COUNTY,ARIZONA,4,NORTHERN,10,APACHE,...,NaN,NaN,0,NaN,CATTLE,"CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURE...",TOTAL,NOT SPECIFIED,(D),(D)


In [5]:
print (slaughter_Q1["data_item"].unique())
print (slaughter_Q1["domain_category"].unique())

['CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURED IN HEAD']
['NOT SPECIFIED']


In [6]:
sorted(slaughter_Q1.columns)

['ag_district',
 'ag_district_code',
 'commodity',
 'county',
 'county_ansi',
 'cv_(%)',
 'data_item',
 'domain',
 'domain_category',
 'geo_level',
 'period',
 'program',
 'region',
 'state',
 'state_ansi',
 'value',
 'watershed',
 'watershed_code',
 'week_ending',
 'year',
 'zip_code']

In [7]:
print (slaughter_Q1.zip_code.unique())
print (slaughter_Q1.week_ending.unique())
print ()
print (slaughter_Q1.watershed.unique())
print (slaughter_Q1.domain_category.unique())
print ()
print (slaughter_Q1.domain.unique())
print (slaughter_Q1.watershed_code.unique())
print ()
print (slaughter_Q1.watershed.unique())
print (slaughter_Q1.region.unique())
print ()
print (slaughter_Q1.program.unique())
print (slaughter_Q1.period.unique())
print ()
print (slaughter_Q1.geo_level.unique())
print (slaughter_Q1.data_item.unique())

[nan]
[nan]

[nan]
['NOT SPECIFIED']

['TOTAL']
[0]

[nan]
[nan]

['CENSUS']
['YEAR']

['COUNTY']
['CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURED IN HEAD']


In [8]:
bad_cols  = ["watershed", "watershed_code", 
             "domain", "domain_category", 
             "region", "period",
             "week_ending", "zip_code", "program", "geo_level"]

slaughter_Q1.drop(bad_cols, axis="columns", inplace=True)
slaughter_Q1.head(2)

,year,state,state_ansi,ag_district,ag_district_code,county,county_ansi,commodity,data_item,value,cv_(%)
0,2017,ALASKA,2,TANANA VALLEY,10,FAIRBANKS NORTH STAR,NaN,CATTLE,"CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURE...",(D),(D)
1,2017,ARIZONA,4,NORTHERN,10,APACHE,1.0,CATTLE,"CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURE...",(D),(D)


In [11]:
slaughter_Q1['county_ansi'].fillna(666, inplace=True)

slaughter_Q1["state_ansi"] = slaughter_Q1["state_ansi"].astype('int32')
slaughter_Q1["county_ansi"] = slaughter_Q1["county_ansi"].astype('int32')

slaughter_Q1["state_ansi"] = slaughter_Q1["state_ansi"].astype('str')
slaughter_Q1["county_ansi"] = slaughter_Q1["county_ansi"].astype('str')

slaughter_Q1.state = slaughter_Q1.state.str.title()
slaughter_Q1.county = slaughter_Q1.county.str.title()
slaughter_Q1.ag_district = slaughter_Q1.ag_district.str.title()
slaughter_Q1.head(2)

,year,state,state_ansi,ag_district,ag_district_code,county,county_ansi,commodity,data_item,value,cv_(%)
0,2017,Alaska,2,Tanana Valley,10,Fairbanks North Star,666,CATTLE,"CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURE...",(D),(D)
1,2017,Arizona,4,Northern,10,Apache,1,CATTLE,"CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURE...",(D),(D)


In [12]:
for idx in slaughter_Q1.index:
    if len(slaughter_Q1.loc[idx, "state_ansi"]) == 1:
        slaughter_Q1.loc[idx, "state_ansi"] = "0" + slaughter_Q1.loc[idx, "state_ansi"]
        
    if len(slaughter_Q1.loc[idx, "county_ansi"]) == 1:
        slaughter_Q1.loc[idx, "county_ansi"] = "00" + slaughter_Q1.loc[idx, "county_ansi"]
    elif len(slaughter_Q1.loc[idx, "county_ansi"]) == 2:
        slaughter_Q1.loc[idx, "county_ansi"] = "0" + slaughter_Q1.loc[idx, "county_ansi"]
        
slaughter_Q1.head(2)

,year,state,state_ansi,ag_district,ag_district_code,county,county_ansi,commodity,data_item,value,cv_(%)
0,2017,Alaska,02,Tanana Valley,10,Fairbanks North Star,666,CATTLE,"CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURE...",(D),(D)
1,2017,Arizona,04,Northern,10,Apache,001,CATTLE,"CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURE...",(D),(D)


In [13]:
slaughter_Q1["county_fips"] = slaughter_Q1["state_ansi"] + slaughter_Q1["county_ansi"]
slaughter_Q1.head(2)

,year,state,state_ansi,ag_district,ag_district_code,county,county_ansi,commodity,data_item,value,cv_(%),county_fips
0,2017,Alaska,02,Tanana Valley,10,Fairbanks North Star,666,CATTLE,"CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURE...",(D),(D),02666
1,2017,Arizona,04,Northern,10,Apache,001,CATTLE,"CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURE...",(D),(D),04001


In [14]:
slaughter_Q1.rename(columns={"value": "cattle_on_feed_sale_4_slaughter", 
                             "cv_(%)": "cattle_on_feed_sale_4_slaughter_cv_(%)"}, inplace=True)

In [15]:
print (slaughter_Q1.shape)
slaughter_Q1 = rc.clean_census(df=slaughter_Q1, col_="cattle_on_feed_sale_4_slaughter")
print (slaughter_Q1.shape)

(12043, 12)
(9638, 12)


In [16]:
slaughter_Q1.head(2)

,year,state,state_ansi,ag_district,ag_district_code,county,county_ansi,commodity,data_item,cattle_on_feed_sale_4_slaughter,cattle_on_feed_sale_4_slaughter_cv_(%),county_fips
0,2017,Arizona,04,Northern,10,Coconino,005,CATTLE,"CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURE...",152.0,2.8,04005
1,2017,Arizona,04,Northern,10,Mohave,015,CATTLE,"CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURE...",100.0,2.8,04015


In [18]:
print (slaughter_Q1.commodity.unique())
print (slaughter_Q1.data_item.unique())

['CATTLE']
['CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURED IN HEAD']


In [19]:
slaughter_Q1 = slaughter_Q1[["year", "data_item", 
                             "cattle_on_feed_sale_4_slaughter", 
                             "cattle_on_feed_sale_4_slaughter_cv_(%)",
                             "county_fips"]]
slaughter_Q1.head(2)

,year,data_item,cattle_on_feed_sale_4_slaughter,cattle_on_feed_sale_4_slaughter_cv_(%),county_fips
0,2017,"CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURE...",152.0,2.8,04005
1,2017,"CATTLE, ON FEED - SALES FOR SLAUGHTER, MEASURE...",100.0,2.8,04015


In [20]:
slaughter_Q1.reset_index(drop=True, inplace=True)

In [21]:
filename = reOrganized_dir + "slaughter_Q1.sav"

export_ = {"slaughter_Q1": slaughter_Q1, 
           "source_code" : "clean_slaughter",
           "Author": "HN",
           "Date" : datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

pickle.dump(export_, open(filename, 'wb'))